Minha primeira vez utilizando o Jypyter Notenook, então realizei a instalação no Ubuntu 18.04 LTS com o seguinte comando: sudo apt install jupyter-notebook

1 - Criei uma classe com o nome csvtopg para subir os arquivos em csv para o Postgres.
2 - Antes de rodar o script, configurar o arquivo banco.ini para realizar a conexão no banco de dados.

In [1]:
import psycopg2, os, csv, configparser
import pandas as pd

class CsvToPg:
    def __init__(self, dbname, user, host, port, password):
        self.dbname =  dbname
        self.user = user
        self.host = host
        self.port = port
        self.password = password
        self.conn = psycopg2.connect(f"dbname = '{self.dbname}' user = '{self.user}' host = '{self.host}' port = '{self.port}' password = '{self.password}' ")
        
    def criarTabela(self, nomearquivo):
         with open(nomearquivo, mode='r', encoding='utf-8-sig') as arquivo:
            try:
                leitor = csv.DictReader(arquivo, delimiter=';')
                leitor_list = list(leitor)
                cursor = self.conn.cursor()
                valores = str(dict(leitor_list[0]).keys()).replace('dict_keys','').replace("'","").replace('([','').replace('])','').replace(",", "  varchar,") + ' varchar'
                tabela = str(nomearquivo).replace('.csv','')
                string = f"DROP TABLE IF EXISTS {tabela}; CREATE TABLE {tabela} ({valores})"
                cursor.execute(string)
                self.conn.commit()
            except Exception as e:
                print('Erro ao criar as tabelas:' , e)
            
    def popularTabela(self, caminho, nomearquivo):
        try:
            cursor = self.conn.cursor()
            tabela = str(nomearquivo).replace('.csv','')
            string = f"COPY {tabela} FROM '{caminho}/{nomearquivo}' WITH DELIMITER ';' CSV"
            cursor.execute(string)
            self.conn.commit()
            print(f'Tabela {str(nomearquivo).replace(".csv", "")} criada e populada com sucesso.')
        except Exception as e:
            print('Erro ao popular as tabelas: ', e)       
            
    def consultaSql(self, consulta):
        cursor = self.conn.cursor()
        cursor.execute(consulta)
        self.conn.commit()
        return cursor.fetchall()
    
if __name__ == "__main__":    
    configuracaobd = configparser.ConfigParser()
    configuracaobd.read('banco.ini')
    dbname = configuracaobd['POSTGRES']['dbname']
    user = configuracaobd['POSTGRES']['user']
    host = configuracaobd['POSTGRES']['host']
    port = configuracaobd['POSTGRES']['port']
    password = configuracaobd['POSTGRES']['password']

    cb = CsvToPg(dbname,user,host,port,password)
    caminho = os.getcwd()
    listanarquivos = os.listdir(caminho)
    for i in listanarquivos:
        if '.csv' in i:
            cb.criarTabela(i)
            cb.popularTabela(caminho, i)
        else:
            pass

Tabela Sales_Customer criada e populada com sucesso.
Tabela Sales_SpecialOfferProduct criada e populada com sucesso.
Tabela Person_Person criada e populada com sucesso.
Tabela Production_Product criada e populada com sucesso.
Tabela Sales_SalesOrderHeader criada e populada com sucesso.
Tabela Sales_SalesOrderDetail criada e populada com sucesso.


1 - Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail
pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.

In [4]:
data = pd.DataFrame(cb.consultaSql("SELECT\n"\
                                   "  COUNT(SalesOrderID)\n"\
                                   "FROM sales_salesorderdetail\n"\
                                   "WHERE split_part(linetotal,'.', 1)::BIGINT >= 3\n"\
                                   "AND linetotal <> 'LineTotal';\n"))
data

,0
0,118041


2 - Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct
e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de
OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture)

In [5]:
    data = pd.DataFrame(cb.consultaSql("SELECT \n" \
                                        "  p.name,\n" \
                                        "  p.DaysToManufacture::BIGINT,\n" \
                                        "  sum(o.OrderQty::BIGINT) AS OrderQty\n" \
                                        "FROM Production_Product as p \n" \
                                        "JOIN Sales_SalesOrderDetail as o ON p.productid = o.productid\n" \
                                        "JOIN Sales_SpecialOfferProduct as s ON p.productid = s.productid\n" \
                                        "GROUP BY 1, 2 ORDER BY 3 DESC LIMIT 3;\n"))
    data

,0,1,2
0,"Sport-100 Helmet, Blue",0,33715
1,AWC Logo Cap,0,33244
2,"Sport-100 Helmet, Black",0,32660


3 - Escreva uma query ligando as tabelas Person.Person, Sales.Customer e
Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem
de pedidos efetuados.

In [6]:
data = pd.DataFrame(cb.consultaSql("SELECT \n" \
                                    "  Person_Person.firstname||' '||  Person_Person.middlename || ' ' ||Person_Person.lastname as nome,\n"\
                                    "  count(sales_SalesOrderHeader.customerid) as pedidos_efetuados\n"\
                                    "FROM Person_Person\n"\
                                    "JOIN Sales_Customer ON Person_Person.businessentityid = Sales_Customer.personid\n"\
                                    "JOIN sales_SalesOrderHeader ON sales_SalesOrderHeader.customerid = Sales_Customer.customerid\n"\
                                    "GROUP BY 1 ORDER BY 2 DESC\n"))
data

,0,1
0,Mason D Roberts,28
1,Dalton NULL Perez,28
2,Charles P Jackson,27
3,Nancy E Chapman,27
4,Daniel NULL Davis,27
...,...,...
19096,Gabriel NULL Bryant,1
19097,Armando F Blanco,1
19098,Miguel K Parker,1
19099,Rachael NULL Sanchez,1


4 - Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e
Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e
OrderDate.

In [7]:
data = pd.DataFrame(cb.consultaSql("SELECT \n"\
                                   "  Production_Product.Productid,\n"\
                                   "  sum(OrderQty::int),\n"\
                                   "  OrderDate::timestamp\n"\
                                   "FROM Sales_SalesOrderDetail\n"\
                                   "JOIN Sales_SalesOrderHeader ON  Sales_SalesOrderDetail.salesorderid = Sales_SalesOrderHeader.salesorderid\n"\
                                   "JOIN Production_Product ON Production_Product.Productid = Sales_SalesOrderDetail.Productid\n"\
                                   "GROUP BY 1,3\n"))
data

,0,1,2
0,707,24,2011-05-31
1,707,58,2011-07-01
2,707,55,2011-08-01
3,707,41,2011-08-31
4,707,77,2011-10-01
...,...,...,...
26873,999,3,2014-05-25
26874,999,1,2014-05-27
26875,999,3,2014-05-28
26876,999,1,2014-05-29


5 - Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela
Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante
o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido
decrescente.

In [8]:
data = pd.DataFrame(cb.consultaSql("SELECT\n"\
                                   "  SalesOrderID,\n"\
                                   "  OrderDate,\n"\
                                   "  replace(TotalDue, ',', '.')::real\n"\
                                   "FROM Sales_SalesOrderHeader\n"\
                                   "WHERE salesorderid <> 'SalesOrderID'\n"\
                                   "AND OrderDate BETWEEN '2011-09-01' AND '2011-09-30'\n"\
                                   "AND replace(TotalDue, ',', '.')::real > 1000\n"\
                                   "ORDER BY 3 DESC\n"))
data

,0,1,2
0,44478,2011-09-29 00:00:00.000,3953.99
1,44444,2011-09-24 00:00:00.000,3953.99
2,44445,2011-09-25 00:00:00.000,3953.99
3,44446,2011-09-25 00:00:00.000,3953.99
4,44447,2011-09-26 00:00:00.000,3953.99
...,...,...,...
131,44468,2011-09-28 00:00:00.000,3729.36
132,44378,2011-09-10 00:00:00.000,3729.36
133,44408,2011-09-15 00:00:00.000,3729.36
134,44379,2011-09-10 00:00:00.000,3729.36
